### Import pandas and datasets

In [1]:
import pandas as pd

# Make sure datasets already uploaded on the root project and converted into xlsx
data = pd.read_excel('market-basket.xlsx')
data.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,3.39,17850.0,United Kingdom


### pip install spark

In [2]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark

### Create spark session/context

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark

### FP Growth with minSupport = 0.006 & minConfidence = 0.005

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.ml.fpm import FPGrowth
import pandas

# make sure all column have str type
data.BillNo = data.BillNo.astype(str)
data.Itemname = data.Itemname.astype(str)
data.Quantity = data.Quantity.astype(str)
data.Date = data.Date.astype(str)
data.Price = data.Price.astype(str)
data.CustomerID = data.CustomerID.astype(str)
data.Country = data.Country.astype(str)

sparkdata = spark.createDataFrame(data)
basketdata = sparkdata.dropDuplicates(['BillNo', 'Itemname']).sort("BillNo")
basketdata = basketdata.groupBy("BillNo").agg(F.collect_list("Itemname")).sort("BillNo")

#Frequent Pattern Growth
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.006, minConfidence=0.005) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets


/content/spark-3.0.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+--------------------+----+
|               items|freq|
+--------------------+----+
|[FELTCRAFT BUTTER...| 471|
|[FELTCRAFT BUTTER...| 167|
|[VINTAGE CHRISTMA...| 334|
|[ROCKING HORSE RE...| 245|
|[6 RIBBONS SHIMME...| 198|
|[VICTORIAN SEWING...| 160|
|[PARTY TIME PENCI...| 133|
|[WHITE HANGING HE...|2202|
|[STRAWBERRY FAIRY...| 198|
|[CAMOUFLAGE LED T...| 160|
|[PINK LOVE HEART ...| 133|
|[JUMBO BAG RED RE...|2064|
|[JUMBO BAG RED RE...| 442|
|[JUMBO BAG VINTAG...| 470|
|[JUMBO BAG VINTAG...| 136|
|[JUMBO BAG VINTAG...| 319|
|[JUMBO BAG VINTAG...| 130|
|[JUMBO BAG VINTAG...| 154|
|[JUMBO BAG VINTAG...| 180|
|[JUMBO BAG VINTAG...| 132|
+--------------------+----+
only showing top 20 rows



In [6]:
#Frequent Pattern Growth
fpGrowth2 = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.02, minConfidence=0.01) 
model2 = fpGrowth2.fit(basketdata)

# Display frequent itemsets.
model2.freqItemsets.show()
items = model2.freqItemsets
# Display generated association rules.
model2.associationRules.show()
rules = model2.associationRules
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model2.transform(basketdata).show()
transformed = model2.transform(basketdata)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[FELTCRAFT BUTTER...| 471|
|[WHITE HANGING HE...|2202|
|[JUMBO BAG RED RE...|2064|
|[JUMBO BAG RED RE...| 442|
|[JUMBO BAG VINTAG...| 470|
|[REGENCY CAKESTAN...|1904|
|[NATURAL SLATE CH...| 468|
|     [PARTY BUNTING]|1656|
|[SMALL HEART MEAS...| 467|
|[LUNCH BAG RED RE...|1541|
|[LUNCH BAG RED RE...| 573|
|[TOY TIDY PINK PO...| 466|
|               [nan]|1455|
|  [PARTY METAL SIGN]| 465|
|[ASSORTED COLOUR ...|1431|
|[LOVE HOT WATER B...| 465|
|[SET OF 3 CAKE TI...|1346|
|[4 TRADITIONAL SP...| 464|
|[PACK OF 72 RETRO...|1279|
|[DOORMAT SPOTTY H...| 463|
+--------------------+----+
only showing top 20 rows

+--------------------+--------------------+-------------------+------------------+
|          antecedent|          consequent|         confidence|              lift|
+--------------------+--------------------+-------------------+------------------+
|[PAPER CHAIN KIT ...|[PAPER CHAIN KIT ...| 0.6720098

In [7]:
# CONCLUSION #
# the higher the minsupport, the less number of frequency items that appear 
# because the quantity of each item must have a higher minimum amount

In [8]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

,items,freq
0,[FELTCRAFT BUTTERFLY HEARTS],471
1,[WHITE HANGING HEART T-LIGHT HOLDER],2202
2,[JUMBO BAG RED RETROSPOT],2064
3,"[JUMBO BAG RED RETROSPOT, WHITE HANGING HEART ...",442
4,[JUMBO BAG VINTAGE CHRISTMAS],470


In [9]:
result_pdf.to_excel('result_pdfItemsFreq.xlsx')

In [10]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

,antecedent,consequent,confidence,lift
0,[PAPER CHAIN KIT VINTAGE CHRISTMAS],[PAPER CHAIN KIT 50'S CHRISTMAS],0.672010,12.736439
1,[JUMBO SHOPPER VINTAGE RED PAISLEY],[JUMBO BAG PINK POLKADOT],0.400515,7.164614
2,[JUMBO SHOPPER VINTAGE RED PAISLEY],[JUMBO STORAGE BAG SUKI],0.445969,8.201213
3,[JUMBO SHOPPER VINTAGE RED PAISLEY],[JUMBO BAG RED RETROSPOT],0.579760,6.084951
4,[LUNCH BAG SPACEBOY DESIGN],[LUNCH BAG RED RETROSPOT],0.482127,6.777627


In [11]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

In [12]:
transformed_pdf.to_excel('transformed_pdfCollectListPred.xlsx')